In [41]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [42]:
DATASET_PATH = "C:\\Users\\sanga\\Downloads\\archive (5)\\Indian-Traffic Sign-Dataset\\Images"
IMG_SIZE = 64
NUM_CLASSES = 59

images = []
labels = []

# Load images in correct numerical order
for folder_name in sorted(os.listdir(DATASET_PATH), key=lambda x: int(x)):
    folder_path = os.path.join(DATASET_PATH, folder_name)
    label = int(folder_name)
    
    for img_file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_file)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
            labels.append(label)


X = np.array(images) / 255.0  # Normalize
y = to_categorical(np.array(labels), num_classes=NUM_CLASSES)


In [43]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [44]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64, (3,3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])


In [45]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [46]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False
)
datagen.fit(X_train)


In [47]:
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/30
350/350 ━━━━━━━━━━━━━━━━━━━━ 15s 38ms/step - accuracy: 0.1950 - loss: 3.2360 - val_accuracy: 0.6200 - val_loss: 1.3284
Epoch 2/30
350/350 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - accuracy: 0.5838 - loss: 1.3976 - val_accuracy: 0.7449 - val_loss: 0.8458
Epoch 3/30
350/350 ━━━━━━━━━━━━━━━━━━━━ 12s 34ms/step - accuracy: 0.7272 - loss: 0.8657 - val_accuracy: 0.7803 - val_loss: 0.6720
Epoch 4/30
350/350 ━━━━━━━━━━━━━━━━━━━━ 13s 37ms/step - accuracy: 0.7820 - loss: 0.6608 - val_accuracy: 0.8018 - val_loss: 0.6059
Epoch 5/30
350/350 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - accuracy: 0.8176 - loss: 0.5228 - val_accuracy: 0.8157 - val_loss: 0.5428
Epoch 6/30
350/350 ━━━━━━━━━━━━━━━━━━━━ 12s 35ms/step - accuracy: 0.8316 - loss: 0.4725 - val_accuracy: 0.8218 - val_loss: 0.5389
Epoch 7/30
350/350 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - accuracy: 0.8494 - loss: 0.4256 - val_accuracy: 0.8265 - val_loss: 0.5309
Epoch 8/30
350/350 ━━━━━━━━━━━━━━━━━━━━ 12s 35ms/step - accuracy: 0.8723 - loss: 0.3521 - 

In [48]:
model.save("traffic_sign_model.h5")


In [ ]:
import cv2
import numpy as np
import pyttsx3
from tensorflow.keras.models import load_model
import time


model = load_model("traffic_sign_model.h5")
IMG_SIZE = 64


class_labels = [
    "Give way",                          # 0
    "No entry",                          # 1
    "One-way traffic",                   # 2
    "One-way traffic",                   # 3 (same as 2)
    "No vehicles in both directions",    # 4
    "No entry for cycles",               # 5
    "No entry for goods vehicles",       # 6
    "No entry for pedestrians",          # 7
    "No entry for bullock carts",        # 8
    "No entry for hand carts",           # 9
    "No entry for motor vehicles",       #10
    "Height limit",                      #11
    "Weight limit",                      #12
    "Axle weight limit",                 #13
    "Length limit",                      #14
    "No left turn",                      #15
    "No right turn",                     #16
    "No overtaking",                     #17
    "Maximum speed limit (90 km/h)",     #18
    "Maximum speed limit (110 km/h)",    #19
    "Horn prohibited",                   #20
    "No parking",                        #21
    "No stopping",                       #22
    "Turn left",                         #23
    "Turn right",                        #24
    "Steep descent",                     #25
    "Steep ascent",                      #26
    "Narrow road",                       #27
    "Narrow bridge",                     #28
    "Unprotected quay",                  #29
    "Road hump",                         #30
    "Dip",                               #31
    "Loose gravel",                      #32
    "Falling rocks",                     #33
    "Cattle",                            #34
    "Crossroads",                        #35
    "Side road junction",                #36
    "Side road junction",                #37 (again)
    "Oblique side road junction",        #38
    "Oblique side road junction",        #39
    "T-junction",                        #40
    "Y-junction",                        #41
    "Staggered side road junction",      #42
    "Staggered side road junction",      #43
    "Roundabout",                        #44
    "Guarded level crossing ahead",      #45
    "Unguarded level crossing ahead",    #46
    "Level crossing countdown marker",   #47
    "Level crossing countdown marker",   #48
    "Level crossing countdown marker",   #49
    "Level crossing countdown marker",   #50
    "Parking",                           #51
    "Bus stop",                          #52
    "First aid post",                    #53
    "Telephone",                         #54
    "Filling station",                   #55
    "Hotel",                             #56
    "Restaurant",                        #57
    "Refreshments"                       #58
]




engine = pyttsx3.init()
engine.setProperty('rate', 130)  

cap = cv2.VideoCapture(0)

last_prediction = None
stable_prediction = None
prediction_start_time = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    
    roi = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    roi_input = np.expand_dims(roi / 255.0, axis=0)


    pred = model.predict(roi_input, verbose=0)
    class_id = np.argmax(pred)
    confidence = np.max(pred)

    if confidence > 0.85:
        predicted_class = class_labels[class_id]

        
        cv2.putText(frame, f"{predicted_class} ({confidence*100:.1f}%)", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        
        if predicted_class != stable_prediction:
            stable_prediction = predicted_class
            prediction_start_time = time.time()
        else:
            
            if time.time() - prediction_start_time >= 1.0 and predicted_class != last_prediction:
                print(f"Detected: {predicted_class}")
                engine.say(f"{predicted_class} detected")
                engine.runAndWait()
                last_prediction = predicted_class
    else:
        stable_prediction = None
        prediction_start_time = 0
        cv2.putText(frame, "Detecting...", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    
    cv2.imshow("Traffic Sign Detection", frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Detected: Telephone
Detected: Restaurant
Detected: Give way
